In [1]:
import pickle

import open3d as o3d
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
with open("data/sample_000000.pkl", "rb") as f:
    sample = pickle.load(f)

In [3]:
points = sample['points']
print("Points shape:", points.shape)
print("Frame ID:", sample['frame_id'])
print("Sample keys:", sample.keys())
print(sample['points'])

Points shape: (16384, 4)
Frame ID: 000000
Sample keys: dict_keys(['frame_id', 'gt_boxes', 'points', 'flip_x', 'noise_rot', 'noise_scale', 'lidar_aug_matrix', 'use_lead_xyz', 'image_shape'])
[[15.322624   -0.9945517  -1.3005242   0.37      ]
 [16.28796     5.4508047  -0.45010167  0.32      ]
 [11.225035    7.8189     -1.5432059   0.21      ]
 ...
 [10.327182    5.269324   -1.5712075   0.4       ]
 [13.737979   -5.4365177  -1.3347485   0.22      ]
 [12.59426    10.674768    0.26653486  0.27      ]]


In [4]:
CLASS_COLORS = {
    1: (1, 0, 0),    # Car     (red)
    2: (0, 1, 0),    # Pedestrian (green)
    3: (0, 0, 1)     # Cyclist (blue)
}

In [6]:
points = sample['points'][:, :3]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

z = points[:, 2]
z_norm = (z - z.min()) / (z.max() - z.min() + 1e-6)
colors = plt.cm.jet(z_norm)[:, :3]
pcd.colors = o3d.utility.Vector3dVector(colors)

gt_boxes = sample['gt_boxes']
boxes = []
for box in gt_boxes:
    x, y, z, dx, dy, dz, heading, class_idx = box
    class_idx = int(class_idx)
    obb = o3d.geometry.OrientedBoundingBox(
        center=[0, 0, 0],
        R=o3d.geometry.get_rotation_matrix_from_xyz([0, 0, heading]),
        extent=[dx, dy, dz]
    )
    obb.translate([x, y, z])
    obb.color = CLASS_COLORS.get(class_idx, (1, 1, 1))
    boxes.append(obb)

o3d.visualization.draw_geometries([pcd, *boxes])